# **EECE-699 Genetic Algorithm on Cart-Pole Environment**

### Install rendering dependencies

In [69]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!apt-get update > /dev/null 2>&1# without update the xvfb library will not work properly
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

/bin/bash: 1#: ambiguous redirect


### Install Gym dependencies

In [70]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

### Import libraries and setup a virtual display for rendering OpenAI gym environments on Google colab. 

In [71]:
import gym
import random
import numpy as np
from random import randint
from statistics import mean, median

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay



In [72]:
display = Display(visible=0, size=(400, 300))
display.start()
env = gym.make("CartPole-v0")
s0 = env.reset()

### GA Implementation

#### Initialize GA parameters

In [73]:
uniform_min = -2
uniform_max = 2
pop_size = 8
n_collect = 10000
th_score = 100
num_parents = 4
crossover_rate = 0.6
num_offsprings = pop_size-num_parents
mutation_rate = 0.1
num_generations = 50
goal_steps = 300 # 200 for v0, 500 for v1

#### Create initial population

In [74]:
def create_initial_pop(pop_size):
    population = []
    for _ in range(pop_size):
        c1 = np.random.uniform(uniform_min, uniform_max) #  cart position
        c2 = np.random.uniform(uniform_min, uniform_max) # cart velocity
        c3 = np.random.uniform(uniform_min, uniform_max) # pole angle
        c4 = np.random.uniform(uniform_min, uniform_max) # pole velocity at tip
        population.append([c1, c2, c3, c4])
    return population

#### Implement the fitness function

In [75]:
def call_fitness(population, S, y):
    fitness = np.empty((len(population), 1))
    for i, pop in enumerate(population):
        mult = (S@(np.array(pop)).T).reshape(-1, 1)
        mult[mult<0] = 0
        mult[mult>0] = 1
        su = np.sum((y == mult))
        fitness[i][0] = su
    return fitness

#### Define selection strategy

In [76]:
def selection(population, fitness, num_parents):
    selected = []
    for _ in range(num_parents):
        # parent = max(fitness)
        # parent_index = fitness.index(parent)
        parent_index = np.argmax(fitness)
        parent_chromosome = population[parent_index]
        fitness[parent_index] = -9999999999
        selected.append(parent_chromosome)
    return selected 

#### Define crossover strategy

In [77]:
def crossover(parents, num_offsprings):
    offsprings = []
    for _ in range(num_offsprings):
        offspring = []
        parent1 = np.random.randint(0, len(parents)) 
        parent2 = np.random.randint(0, len(parents))
        while (parent2 == parent1): parent2 = np.random.randint(0, len(parents))
        parent1 = parents[parent1] 
        parent2 = parents[parent2] 
        for i in range(len(parents[0])):
            if np.random.rand() > crossover_rate:
                offspring.append(parent2[i])
            else:
                offspring.append(parent1[i])
        offsprings.append(offspring)
    return offsprings

#### Define mutation strategy

In [78]:
def mutation(offsprings):
    for i in range(len(offsprings)):
        if np.random.rand() < mutation_rate:
            chromosome = np.random.randint(0, 4)
            noise = np.random.uniform(-1, 1)
            offsprings[i][chromosome] += noise
    return offsprings

#### Full GA algorithm
P.S. you can change the parameter in your implementation or define a new GA_model() function


In [91]:
def create_data():
    training_data, accepted_scores = [], []
    for i in range(n_collect):
        env.reset()        
        score = 0
        game_memory, prev_observation = [], []
        for _ in range(goal_steps):
            action = random.randint(0,1)
            observation, reward, done, info = env.step(action)
            if done:
                break
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score += reward

        if score >= th_score:
            accepted_scores.append(score)
            for data in game_memory:
                training_data.append(data)         
       
    print(f'Number of accepted instances {len(training_data)}')  
    print(f'Average accepted score: {mean(accepted_scores)}')

    return training_data

training_data  = create_data()
fitness_threshold = len(training_data)

0
1000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

In [80]:
def GA_model(training_data):
    X = np.array([e[0] for e in training_data])
    y = np.array([e[1] for e in training_data]).reshape(-1, 1)
    
    population = create_initial_pop(pop_size)
    fitness = call_fitness(population, X, y)

    for i in range(num_generations):
        parents = selection(population, fitness, num_parents)
        offsprings = crossover(parents, num_offsprings)
        mutants = mutation(offsprings)
        population = []
        population.extend(parents)
        population.extend(mutants)
        fitness = call_fitness(population, X, y)
        if np.max(fitness) > fitness_threshold:
            break
    best_pop_index = np.argmax(np.max(fitness))
    weights= [population[best_pop_index]]

    return weights

In [81]:
weights = GA_model(training_data)
weights = np.asarray(weights)
print(f'Weights: {weights}')


Weights: [[-2.09883437  0.94142699  0.25770885  1.18978978]]


### GA Testing
Run your GA model and print average or final scores achieved

In [82]:
def GA_model_predict(test_data, weights):
    hx = test_data@(weights).T
    return int(hx > 0)

In [83]:
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

env = wrap_env(gym.make("CartPole-v0"))


In [84]:
scores = []
for _ in range(1000):
    score = 0
    env.reset()
    for i in range(goal_steps):
        if i == 0:
            action = random.randint(0,1)
        else:
            action = GA_model_predict(prev_obs, weights)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score += reward
        if done:
            break
    scores.append(score)

env.close()
print(f"Average score is {sum(scores)/len(scores)}")

Average score is 130.77


In [85]:
score = 0
env.reset()
for i in range(goal_steps):
    env.render()
    if i == 0:
        action = random.randint(0,1)
    else:
        action = GA_model_predict(prev_obs, weights)
    new_observation, reward, done, info = env.step(action)
    prev_obs = new_observation
    score += reward
    if done:
        break
print(f"Final score achieved is {score}")
show_video()
env.close()


Final score achieved is 93.0
